<a href="https://colab.research.google.com/github/nithyasri0913/Movie-Recommendation-System/blob/main/movie_recommendation_system_content_popularity_based_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Data collection and pre-processing


In [36]:
#loading the data
movies_data=pd.read_csv('/movies.csv')

In [37]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [38]:
movies_data.shape

(4803, 24)

In [39]:
#feature selection
selected_features=['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [40]:
for feature in selected_features:
  movies_data[feature]=movies_data[feature].fillna('')

In [41]:
#combining features
combined_features=movies_data['genres']+movies_data['keywords']+movies_data['tagline']+movies_data['cast']+movies_data['director']
combined_features.head()

0    Action Adventure Fantasy Science Fictioncultur...
1    Adventure Fantasy Actionocean drug abuse exoti...
2    Action Adventure Crimespy based on novel secre...
3    Action Crime Drama Thrillerdc comics crime fig...
4    Action Adventure Science Fictionbased on novel...
dtype: object

In [42]:
#converting text to feature vectors
vectorizer=TfidfVectorizer()

In [43]:
feature_vectors=vectorizer.fit_transform(combined_features)
print(feature_vectors)

  (0, 3583)	0.16941894714909375
  (0, 20729)	0.2738578969459924
  (0, 16614)	0.15696073877453268
  (0, 14550)	0.2236809363648068
  (0, 23267)	0.16128139780622516
  (0, 26547)	0.19638671147741732
  (0, 22464)	0.20207089568711745
  (0, 21487)	0.21550128478931546
  (0, 27540)	0.19771357974524176
  (0, 27182)	0.23480088356130552
  (0, 21521)	0.1562356665943368
  (0, 18558)	0.2612168335104853
  (0, 18164)	0.08690831799482265
  (0, 27161)	0.1261988657937151
  (0, 24197)	0.07518543993419265
  (0, 22778)	0.2738578969459924
  (0, 4676)	0.2452909770424961
  (0, 26334)	0.13059723071916296
  (0, 22916)	0.33668756406929184
  (0, 10193)	0.16532432420812987
  (0, 4456)	0.21799675215510664
  (0, 9304)	0.2738578969459924
  (0, 21836)	0.09966592997173944
  (0, 8936)	0.11806131645084653
  (0, 444)	0.09109355212252294
  :	:
  (4801, 5449)	0.31510541707008866
  (4801, 22170)	0.31510541707008866
  (4801, 27361)	0.31510541707008866
  (4801, 27513)	0.3005604008026175
  (4801, 8121)	0.2611505149930138
  (4801,

### Cosine Similarity

In [44]:
similarity=cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.06865296 0.01492221 ... 0.         0.         0.        ]
 [0.06865296 1.         0.02799128 ... 0.01243107 0.         0.        ]
 [0.01492221 0.02799128 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.01243107 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [45]:
similarity.shape

(4803, 4803)

In [46]:
#getting movie name from the user
movie_name=input("Enter the movie name: ")

Enter the movie name: The Godfather


In [49]:
#list of all movies
all_titles=movies_data['title'].tolist()
#print(all_titles)

In [50]:
#find the closest matches
find_close_match=difflib.get_close_matches(movie_name,all_titles)
print(find_close_match)

['The Godfather', 'The Last Godfather', 'The Watcher']


In [51]:
close_match=find_close_match[0]
print(close_match)

The Godfather


In [52]:
#index of the movie with title
index=movies_data[movies_data.title==close_match]['index'].values[0]
print(index)

3337


In [53]:
# list of similar movies
similarity_score=list(enumerate(similarity[index]))
#print(similarity_score)

In [54]:
#sort based on similarity score
sorted_score=sorted(similarity_score,key=lambda x:x[1],reverse=True)
#print(sorted_score)

In [55]:
#print the name of similar movies based on index
print('Suggestions: \n')
i=1
for movie in sorted_score:
  index=movie[0]
  title_from_index=movies_data[movies_data.index==index]['title'].values[0]
  if(i<=10):
    print(i,')',title_from_index)
    i=i+1


Suggestions: 

1 ) The Godfather
2 ) Closer
3 ) Apocalypse Now
4 ) The Godfather: Part II
5 ) The Godfather: Part III
6 ) August Rush
7 ) American Graffiti
8 ) Machete
9 ) Leaving Las Vegas
10 ) West Side Story


# Movie Recommender System

In [56]:
movie_name=input("Enter the movie name: ")
#list of all movies
all_titles=movies_data['title'].tolist()
#find the closest matches
find_close_match=difflib.get_close_matches(movie_name,all_titles)
close_match=find_close_match[0]
#index of the movie with title
index=movies_data[movies_data.title==close_match]['index'].values[0]
# list of similar movies
similarity_score=list(enumerate(similarity[index]))
#sort based on similarity score
sorted_score=sorted(similarity_score,key=lambda x:x[1],reverse=True)
#print the name of similar movies based on index
print('Suggestions: \n')
i=1
for movie in sorted_score:
  index=movie[0]
  title_from_index=movies_data[movies_data.index==index]['title'].values[0]
  if(i<=10):
    print(i,')',title_from_index)
    i=i+1


Enter the movie name: The Godfather
Suggestions: 

1 ) The Godfather
2 ) Closer
3 ) Apocalypse Now
4 ) The Godfather: Part II
5 ) The Godfather: Part III
6 ) August Rush
7 ) American Graffiti
8 ) Machete
9 ) Leaving Las Vegas
10 ) West Side Story
